<a href="https://colab.research.google.com/github/nano-bot01/Movie-Recommendation-System-for-Hollywoord-movies/blob/main/Movie_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommendation system 

## Importing the dependencies

In [325]:
import numpy as np
import pandas as pd
import difflib
from scipy.sparse import find

from sklearn.feature_extraction.text import TfidfVectorizer 
# for transforming the textual data to numerical feature vector to find present similarity values 
from sklearn.metrics.pairwise import cosine_similarity

## Dataset Loading and Preprocessing


In [326]:
# loading the dataset

dataset = pd.read_csv('movies.csv')

In [327]:
dataset.head()

,Index,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,1,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,2,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,3,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,4,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,5,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [328]:
dataset.tail()

,Index,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
995,996,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
996,997,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
997,998,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
998,999,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN
999,1000,https://m.media-amazon.com/images/M/MV5BMTY5OD...,The 39 Steps,1935,NaN,86 min,"Crime, Mystery, Thriller",7.6,A man in London tries to help a counter-espion...,93.0,Alfred Hitchcock,Robert Donat,Madeleine Carroll,Lucie Mannheim,Godfrey Tearle,51853,NaN


In [329]:
# columns in dataset

dataset.columns

Index(['Index', 'Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

In [330]:
# shape of dataset frame

dataset.shape

(1000, 17)

In [331]:
# information about dataset

dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Index          1000 non-null   int64  
 1   Poster_Link    1000 non-null   object 
 2   Series_Title   1000 non-null   object 
 3   Released_Year  1000 non-null   object 
 4   Certificate    899 non-null    object 
 5   Runtime        1000 non-null   object 
 6   Genre          1000 non-null   object 
 7   IMDB_Rating    1000 non-null   float64
 8   Overview       1000 non-null   object 
 9   Meta_score     843 non-null    float64
 10  Director       1000 non-null   object 
 11  Star1          1000 non-null   object 
 12  Star2          1000 non-null   object 
 13  Star3          1000 non-null   object 
 14  Star4          1000 non-null   object 
 15  No_of_Votes    1000 non-null   int64  
 16  Gross          831 non-null    object 
dtypes: float64(2), int64(2), object(13)
memory usage: 132

In [332]:
# checking for null values in dataset

dataset.isnull().sum()

Index              0
Poster_Link        0
Series_Title       0
Released_Year      0
Certificate      101
Runtime            0
Genre              0
IMDB_Rating        0
Overview           0
Meta_score       157
Director           0
Star1              0
Star2              0
Star3              0
Star4              0
No_of_Votes        0
Gross            169
dtype: int64

In [333]:
# Total count of null values

dataset.isnull().sum().sum()

427

In [334]:
# changing the title of movies to lowercase 

dataset['Series_Title'] = dataset['Series_Title'].str.lower()
print(dataset['Series_Title'].head())

0    the shawshank redemption
1               the godfather
2             the dark knight
3      the godfather: part ii
4                12 angry men
Name: Series_Title, dtype: object


In [335]:
selected_features = ['Genre', 'Certificate', 'Overview', 'Star1','Runtime', "Director"]
print(selected_features)

['Genre', 'Certificate', 'Overview', 'Star1', 'Runtime', 'Director']


In [336]:
# replacing the null values with null string

for features in selected_features :
  dataset[features] = dataset[features].fillna('')

### Combining the selected features 

In [337]:
dataset['Runtime'] = dataset['Runtime'].astype(str)

In [338]:
combined_features = dataset['Genre']+''+dataset['Certificate']+''+dataset['Overview']+''+dataset['Star1']+''+dataset['Runtime']+''+dataset['Director']

In [339]:
combined_features.head()

0    DramaATwo imprisoned men bond over a number of...
1    Crime, DramaAAn organized crime dynasty's agin...
2    Action, Crime, DramaUAWhen the menace known as...
3    Crime, DramaAThe early life and career of Vito...
4    Crime, DramaUA jury holdout attempts to preven...
dtype: object

In [340]:
print(combined_features )

0      DramaATwo imprisoned men bond over a number of...
1      Crime, DramaAAn organized crime dynasty's agin...
2      Action, Crime, DramaUAWhen the menace known as...
3      Crime, DramaAThe early life and career of Vito...
4      Crime, DramaUA jury holdout attempts to preven...
                             ...                        
995    Comedy, Drama, RomanceAA young New York social...
996    Drama, WesternGSprawling epic covering the lif...
997    Drama, Romance, WarPassedIn Hawaii in 1941, a ...
998    Drama, WarSeveral survivors of a torpedoed mer...
999    Crime, Mystery, ThrillerA man in London tries ...
Length: 1000, dtype: object


Converting the text data into features vector

In [341]:
vectorizer = TfidfVectorizer()
feature_vector = vectorizer.fit_transform(combined_features )

In [342]:
print(feature_vector)

  (0, 1708)	0.2505148726124533
  (0, 4807)	0.21443452396919715
  (0, 6454)	0.2654301317345659
  (0, 7639)	0.23993232785188154
  (0, 1764)	0.2654301317345659
  (0, 1404)	0.23993232785188154
  (0, 184)	0.2505148726124533
  (0, 7624)	0.1725198012309844
  (0, 6241)	0.21010180960765623
  (0, 2435)	0.2654301317345659
  (0, 377)	0.05942101324341712
  (0, 7032)	0.2505148726124533
  (0, 2718)	0.2250170687297689
  (0, 8349)	0.1561617863995958
  (0, 5509)	0.11630253532333261
  (0, 5471)	0.2505148726124533
  (0, 5604)	0.1772222227769947
  (0, 848)	0.2027200266596791
  (0, 4627)	0.1710770455217085
  (0, 3619)	0.21443452396919715
  (0, 2064)	0.2505148726124533
  (1, 1515)	0.2251568633942768
  (1, 2805)	0.2048043690714165
  (1, 4806)	0.22011472136437507
  (1, 902)	0.2724611616031378
  :	:
  (999, 4712)	0.1550641804810998
  (999, 6599)	0.1531399002726865
  (999, 3379)	0.13341964658812505
  (999, 8194)	0.10999857145130192
  (999, 8197)	0.14105055473172587
  (999, 7760)	0.13540811626108384
  (999, 5249)

### Cosine Similiarity 

In [343]:
# similarity confidence value using cosine similiarity

similarity = cosine_similarity(feature_vector)


In [344]:
print(similarity)

[[1.         0.00694229 0.02290816 ... 0.0031684  0.01806135 0.00572377]
 [0.00694229 1.         0.03689459 ... 0.01952957 0.00926989 0.02379964]
 [0.02290816 0.03689459 1.         ... 0.02136934 0.06260187 0.0667727 ]
 ...
 [0.0031684  0.01952957 0.02136934 ... 1.         0.02758111 0.03654292]
 [0.01806135 0.00926989 0.06260187 ... 0.02758111 1.         0.07787157]
 [0.00572377 0.02379964 0.0667727  ... 0.03654292 0.07787157 1.        ]]


In [345]:
print(similarity.shape)
# compare with all the other movies present there

(1000, 1000)


In [346]:
# creating a list with all the moviee names in dataset

list_of_all_titles = dataset['Series_Title'].tolist()
print(list_of_all_titles)

['the shawshank redemption', 'the godfather', 'the dark knight', 'the godfather: part ii', '12 angry men', 'the lord of the rings: the return of the king', 'pulp fiction', "schindler's list", 'inception', 'fight club', 'the lord of the rings: the fellowship of the ring', 'forrest gump', 'il buono, il brutto, il cattivo', 'the lord of the rings: the two towers', 'the matrix', 'goodfellas', 'star wars: episode v - the empire strikes back', "one flew over the cuckoo's nest", 'hamilton', 'gisaengchung', 'soorarai pottru', 'interstellar', 'cidade de deus', 'sen to chihiro no kamikakushi', 'saving private ryan', 'the green mile', 'la vita è bella', 'se7en', 'the silence of the lambs', 'star wars', 'seppuku', 'shichinin no samurai', "it's a wonderful life", 'joker', 'whiplash', 'the intouchables', 'the prestige', 'the departed', 'the pianist', 'gladiator', 'american history x', 'the usual suspects', 'léon', 'the lion king', 'terminator 2: judgment day', 'nuovo cinema paradiso', 'hotaru no hak

## Recommedation

In [347]:
# getting the name as input for recommendation

movie_name = input('Enter your favourite movie name : ')

In [348]:
# finding the movie name given by the user

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
print(find_close_match)

['interstellar', 'the wrestler', 'the hustler']


In [349]:
close_match = find_close_match[0]
print(close_match)

interstellar


In [350]:
# Index of movie with title and data

index_of_movie = dataset[dataset.Series_Title == close_match]['Index'].values[0]
# we are finding the index of movie by comparing the title of each movie with closly matched string and returing its index 
print(index_of_movie)

22


In [351]:
# finding similariy score 

score = list(enumerate(similarity[index_of_movie]))
print(score)

[(0, 0.010591116886424928), (1, 0.026983010999941468), (2, 0.07417067422367887), (3, 0.03298816165265901), (4, 0.02653499981995562), (5, 0.05578942517311759), (6, 0.05019105497319811), (7, 0.0050656632075965526), (8, 0.02470536898123979), (9, 0.0031939648265096746), (10, 0.044856492118687086), (11, 0.04812818808121332), (12, 0.020529456947039598), (13, 0.0287399424464006), (14, 0.019936530550226393), (15, 0.03599261492224309), (16, 0.019058243535115267), (17, 0.020739786499837924), (18, 0.03961405190780181), (19, 0.02132907686416726), (20, 0.02457815261710818), (21, 0.007510880453691342), (22, 1.0), (23, 0.014238996130843899), (24, 0.012487722592934462), (25, 0.04374935784065049), (26, 0.04853949134107072), (27, 0.035210443013151196), (28, 0.02297503704259558), (29, 0.03562630621390407), (30, 0.016049302422171088), (31, 0.00367414408472386), (32, 0.003013423302044658), (33, 0.02938978066793898), (34, 0.005633284866494162), (35, 0.00923052626308921), (36, 0.05252193503258639), (37, 0.05

In [352]:
len(score)

1000

In [353]:
# finding movies having higher similarity score 

sorted_list = sorted(score, key = lambda x:x[1], reverse = True)
# sort based on non increasing order, key is based on first index of score in list element which is score in (index, score)
print(sorted_list)

[(22, 1.0), (946, 0.12023786933315404), (81, 0.09038913488268187), (993, 0.08657802495732932), (268, 0.0850347219541837), (668, 0.08478911119358792), (232, 0.08458898968128904), (710, 0.0786758440484397), (662, 0.0765742057406775), (471, 0.07617641971716277), (108, 0.07580821909651576), (2, 0.07417067422367887), (509, 0.07206975760820584), (474, 0.07186326161592219), (356, 0.07147457078890923), (162, 0.07052857564501783), (704, 0.0696857310830589), (933, 0.06821302256346118), (780, 0.06810865853379489), (550, 0.06732331351411841), (521, 0.06582354679977057), (181, 0.06509432787520979), (270, 0.064180254862224), (600, 0.06359036075343714), (319, 0.0635436895586379), (38, 0.06317563672919285), (934, 0.0628511543578174), (935, 0.06222348275211493), (909, 0.06204059585054428), (959, 0.060663678806485925), (737, 0.060033242463842236), (311, 0.0594774629458338), (262, 0.05874971885712505), (418, 0.05829762144855852), (51, 0.05778614395514732), (316, 0.05768320331776114), (872, 0.057315598276

In [354]:
print(sorted_list[0], sorted_list[1], sorted_list[2], sorted_list[3], sorted_list[4])
# top five score with index 

(22, 1.0) (946, 0.12023786933315404) (81, 0.09038913488268187) (993, 0.08657802495732932) (268, 0.0850347219541837)


In [355]:
# printing the name of simila rmovies based on the index

print("Movies Results : \n")

i = 1

for movie in sorted_list:
  index = movie[0]
  title_of_movie = dataset[dataset.index == index]['Series_Title'].values[0]
  if(i<=5):
    print(i, '.', title_of_movie)
    i += 1

Movies Results : 

1 . cidade de deus
2 . y tu mamá también
3 . rear window
4 . blowup
5 . platoon


## Recommendation function 

In [356]:
def recommend(name):
  find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)
  
  try:
    close_match = find_close_match[0]
  except IndexError:
    close_match = 'null'
    print("No result found with this title")
    return 
  index_of_movie = dataset[dataset.Series_Title == close_match]['Index'].values[0]
  score = list(enumerate(similarity[index_of_movie]))
  sorted_list = sorted(score, key = lambda x:x[1], reverse = True)


  print("Movies Results : \n")

  i = 1

  for movie in sorted_list:
    index = movie[0]
    title_of_movie = dataset[dataset.index == index]['Series_Title'].values[0]
    if(i<=5):
      print(i, '. ', title_of_movie)
      i += 1

In [357]:
movie_name = input('Enter your favourite movie name : ')

recommend(movie_name)

Movies Results : 

1 .  cidade de deus
2 .  y tu mamá también
3 .  rear window
4 .  blowup
5 .  platoon
